In [34]:
import cv2
import numpy as np
import torch,torchvision
from torchvision import transforms


model= torch.load('model.pt')




load_from_sys = True

if load_from_sys:
	hsv_value = np.load('hsv_value.npy')       #value=array([[  0,  59, 253], [ 87, 255, 255]], dtype=int64)

cap = cv2.VideoCapture(0)
cap.set(3,1280)                            #width,resolution
cap.set(4,720)



#loading the model

def prediction(image,model):
    
    T=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    
    image=np.resize(28,28)
    x=(255-np.expand_dims(np.array(image),axis=-1))/255.
    x=T(x)
    with torch.no_grad():
        pred=model(x)
        return argmax(pred)


kernel = np.ones((5,5), np.uint8)            #filter to smoothen the image

canvas = None

x1 = 0
y1 = 0

noise_thresh = 800                     #thrshold value


while True:
	_, frame = cap.read()                #boolean value,frame read
	frame = cv2.flip(frame, 1)            # 1 means flipping arond y axis

	if canvas is None:
		canvas = np.zeros_like(frame)         #creating black screen of same size

	hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)       #converting to hsv

	if load_from_sys:
		lower_range = hsv_value[0]               #0,59,253
		upper_range = hsv_value[1]               #87,255,255

	mask = cv2.inRange(frame,lower_range, upper_range)     #mask of neon orange

	mask = cv2.erode(mask, kernel, iterations = 1)
	mask = cv2.dilate(mask, kernel, iterations = 2)          #erosion and dilation(processing of mask)

	contours, heirarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)      
                   #chain stores x,y of end points
                   #rect external only stores eldest,ie,outermost
        
	if contours  and cv2.contourArea(max(contours, key = cv2.contourArea)) > noise_thresh:
		c = max(contours, key = cv2.contourArea)
		x2, y2 ,w, h = cv2.boundingRect(c)                  #creating a bounding box in the mask

		if x1 == 0 and y1 == 0:
			x1,y1 = x2,y2
		else:
			canvas = cv2.line(canvas, (x1,y1), (x2,y2), [0,255,255], 4)    #creating a green/redline in canvas thickness 4 

		x1,y1 = x2,y2
	
	else:
		x1,y1 = 0, 0
        
    
	frame = cv2.add(frame, canvas)

	stacked = np.hstack((canvas, frame))                 #to display the canvas and frame together
	cv2.imshow('Screen_Pen', cv2.resize(stacked, None, fx = 0.6, fy = 0.6))     #minimizing th imagw

	if cv2.waitKey(1) == 13:
		break

	#Clear the canvas when 'c' is pressed
    
	if cv2.waitKey(1) & 0xFF == ord('c'):           
		canvas = None

	if cv2.waitKey(1) & 0xFF == ord('p'):
		a=prediction(canvas,model)
		cv2.putText(canvas, "Convolution Neural Network:  " + str(a), (20, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
					255,3)  
        
        #image,text,coordinates,dtyle,thickness,colour,linetype
        

        
cv2.destroyAllWindows()
cap.release()

RuntimeError: Input type (double) and bias type (float) should be the same